In [1]:
from tqdm import tqdm
import torch
import json
import numpy as np
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [3]:
device = torch.device('cuda:0')
torch.cuda.empty_cache()

In [4]:
model_name = "i3d_r50"
model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=True)
model = model.to(device)
model = model.eval()

Using cache found in /home/ichuviliaeva/.cache/torch/hub/facebookresearch_pytorchvideo_master


Проверяем, что модель предобучена

In [5]:
%cd /DATA/ichuviliaeva/videos/i3d_experemental

/DATA/ichuviliaeva/videos/i3d_experemental


In [6]:
!wget https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json

--2023-01-26 06:03:56--  https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10326 (10K) [text/plain]
Saving to: ‘kinetics_classnames.json.1’

kinetics_classnames 100%[===================>]  10.08K  --.-KB/s    in 0.001s  

2023-01-26 06:03:57 (14.3 MB/s) - ‘kinetics_classnames.json.1’ saved [10326/10326]



In [7]:
with open("kinetics_classnames.json", "r") as f:
    kinetics_classnames = json.load(f)

kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

In [8]:
print(kinetics_id_to_classname[7])

arranging flowers


In [9]:
import torch.nn as nn
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)

In [10]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
alpha = 4

In [11]:
transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size)
        ]
    ),
)

In [12]:
clip_duration = (num_frames * sampling_rate)/frames_per_second

In [13]:
!wget https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4

--2023-01-26 06:03:57--  https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549197 (536K) [video/mp4]
Saving to: ‘archery.mp4.1’

archery.mp4.1       100%[===================>] 536.33K   614KB/s    in 0.9s    

2023-01-26 06:03:59 (614 KB/s) - ‘archery.mp4.1’ saved [549197/549197]



In [18]:
video_path = "archery.mp4"

start_sec = 0
end_sec = start_sec + clip_duration

video = EncodedVideo.from_path(video_path)

video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

video_data = transform(video_data)

inputs = video_data["video"]
inputs = [i.to(device)[None, ...] for i in inputs]
inputs = torch.cat(inputs).unsqueeze(0)

In [15]:
predicts = model(inputs)

In [16]:
post_act = torch.nn.Softmax(dim=1)
predicts = post_act(predicts)
pred_classes = predicts.topk(k=5)
print(pred_classes)

torch.return_types.topk(
values=tensor([[0.5663, 0.0116, 0.0094, 0.0094, 0.0091]], device='cuda:0',
       grad_fn=<TopkBackward>),
indices=tensor([[  5, 333, 356, 119, 134]], device='cuda:0'))


In [17]:
pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes.indices[0]]
print("Predicted labels: %s" % ", ".join(pred_class_names))

Predicted labels: archery, stretching arm, throwing axe, exercising arm, front raises


да, это было видео про стрельбу из лука -- всё хорошо и модель предобучена